En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:

1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.

2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.


>El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:
>
>* Cargaremos los dos ficheros de datos
>* Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
>    * USA
>    * Australia
>    * New Zealand
>    * South Africa
>    * Papua New Guinea
>
>* Del dataframe de los datos climáticos seleccionaremos todas las columnas.
>* Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
>    * Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
>* Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas.

```python
# os recomendamos resetear el index del dataframe de los datos climáticos para que no se repitan los nombres de las columnas.
​
​
# El primer problema al que nos podemos enfrentar es que si vemos los tipos de las columnas vemos que estas columnas son objetos, es decir, strings, lo que hará que trabajar con ellas sea un poco complicado: 
clima.dtypes
​
timepoint             int64
cloudcover            int64
highcloud             int64
midcloud              int64
lowcloud              int64
rh_profile           object
wind_profile         object
​
# en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 
​
import ast
​
clima['wind_profile']= clima['wind_profile'].apply(ast.literal_eval)
​
# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 
​
​
x = clima['wind_profile'].apply(pd.Series)
​
​
# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
x = df['rh_profile'].apply(pd.Series) 
​
# ¿Qué es lo que ocurre cuando hacemos esto?
# Nos ha creado tantas columnas como valores tuvieramos en la lista. Donde columna es, en este caso, un diccionario (porque nuestra lista esta compuesta por distintos diccionarios)
​
# Ok, hemos conseguido desempaquetar la información de la lista en distintas columnas. Ahora tenemos que despempaquetar la información de los diccionarios en distintas columnas. En este caso, lo que querremos es que las key de los diccionarios sean los nombres de las columnas y los values los valores de las celdas del dataframe. Volveremos a seguir entonces la misma lógica que antes con el apply, pero en este caso necesitamos hacerlo para todo el dataframe (que es x): 
​
# Por eso empezamos con un for para iterar por cada una de las columnas. 
for i in range(len(x.columns)): 
​
    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(x[i].apply(pd.Series)["layer"][0]) 
​
    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    valores = list(x[i].apply(pd.Series)["rh"] )
​
    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    df.insert(i, nombre, valores)
​
# una vez que hayamos hecho esto para las dos columnas ya podremos hacer el gropuby para después unir toda la información. 
```

> * Guardar los resultados obtenidos en un csv que usaremos en próximos ejercicios de pair programming.

##### Importamos librerías

In [1]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np
import ast

pd.options.display.max_columns=None
#pd.options.display.max_rows=5
pd.options.display.max_rows=None

##### Cargamos dataframes

In [2]:
df_attacks = pd.read_csv('datos/input/attacks_limpieza_completa.csv', index_col =0)

In [3]:
df_clima = pd.read_pickle('datos/input/datos_clima_paises.pkl')

In [4]:
df_attacks.head(1)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327


In [5]:
df_clima.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 155, 'speed':...",27,2,2,1011,none,0,0,190,2,usa,39.78373,-100.445882


##### Seleccionamos los paises que nos interesan del df attacks

In [6]:
# Sacamos la lista de paises para los que tenemos los datos climáticos
lista_paises = list(df_clima['pais'].unique())
lista_paises

['usa', 'australia', 'south africa', 'new zealand', 'papua new guinea']

In [7]:
# Filtramos el df attacks por los paises que queremos
df_paises = df_attacks[df_attacks['country'].isin(lista_paises)]
df_paises.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


In [8]:
# Comprobamos
df_paises['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

##### Desempaquetamos las columnas `rh_profile` y `wind_profile` del df clima

In [9]:
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 155, 'speed':...",27,2,2,1011,none,0,0,190,2,usa,39.78373,-100.445882
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 155, 'speed':...",26,2,2,1010,none,0,0,155,2,usa,39.78373,-100.445882


In [10]:
# Creemos que por haber guardado en pkl no nos ha hecho falta hacer ningún casteo (de hecho nos daba error), así que nos ahorramos un paso
df_clima['rh_profile']

0      [{'layer': '950mb', 'rh': 4}, {'layer': '900mb...
1      [{'layer': '950mb', 'rh': 3}, {'layer': '900mb...
2      [{'layer': '950mb', 'rh': 4}, {'layer': '900mb...
3      [{'layer': '950mb', 'rh': 5}, {'layer': '900mb...
4      [{'layer': '950mb', 'rh': 6}, {'layer': '900mb...
5      [{'layer': '950mb', 'rh': 5}, {'layer': '900mb...
6      [{'layer': '950mb', 'rh': 4}, {'layer': '900mb...
7      [{'layer': '950mb', 'rh': 4}, {'layer': '900mb...
8      [{'layer': '950mb', 'rh': 5}, {'layer': '900mb...
9      [{'layer': '950mb', 'rh': 4}, {'layer': '900mb...
10     [{'layer': '950mb', 'rh': 7}, {'layer': '900mb...
11     [{'layer': '950mb', 'rh': 8}, {'layer': '900mb...
12     [{'layer': '950mb', 'rh': 10}, {'layer': '900m...
13     [{'layer': '950mb', 'rh': 10}, {'layer': '900m...
14     [{'layer': '950mb', 'rh': 9}, {'layer': '900mb...
15     [{'layer': '950mb', 'rh': 6}, {'layer': '900mb...
16     [{'layer': '950mb', 'rh': 5}, {'layer': '900mb...
17     [{'layer': '950mb', 'rh'

In [11]:
df_clima['wind_profile']

0      [{'layer': '950mb', 'direction': 155, 'speed':...
1      [{'layer': '950mb', 'direction': 155, 'speed':...
2      [{'layer': '950mb', 'direction': 120, 'speed':...
3      [{'layer': '950mb', 'direction': 110, 'speed':...
4      [{'layer': '950mb', 'direction': 265, 'speed':...
5      [{'layer': '950mb', 'direction': 320, 'speed':...
6      [{'layer': '950mb', 'direction': 335, 'speed':...
7      [{'layer': '950mb', 'direction': 355, 'speed':...
8      [{'layer': '950mb', 'direction': 10, 'speed': ...
9      [{'layer': '950mb', 'direction': 110, 'speed':...
10     [{'layer': '950mb', 'direction': 170, 'speed':...
11     [{'layer': '950mb', 'direction': 205, 'speed':...
12     [{'layer': '950mb', 'direction': 200, 'speed':...
13     [{'layer': '950mb', 'direction': 230, 'speed':...
14     [{'layer': '950mb', 'direction': 240, 'speed':...
15     [{'layer': '950mb', 'direction': 300, 'speed':...
16     [{'layer': '950mb', 'direction': 250, 'speed':...
17     [{'layer': '950mb', 'dir

##### Vamos primero con la columna `rh_profile`

In [12]:
# Entendiendo las series, razonamos y probamos que, cuando le damos una lista a una serie la ordena con indices, que serán las columnas de un dataframe. 
# Los values de la serie serán los elementos de la lista, que serán los registros de una sola de las filas para cada columna de un dataframe.
# Por lo tanto, si aplicamos un pd.Series a cada fila, nos convertirá esa lista en un conjunto de columnas ordenadas por un índice asignado, y cada value será cada valor de la fila. 
df_rh_layer = df_clima['rh_profile'].apply(pd.Series)
df_rh_layer.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 4}","{'layer': '900mb', 'rh': 5}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 7}","{'layer': '750mb', 'rh': 8}","{'layer': '700mb', 'rh': 11}","{'layer': '650mb', 'rh': 11}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 3}","{'layer': '500mb', 'rh': 1}","{'layer': '450mb', 'rh': 6}","{'layer': '400mb', 'rh': 7}","{'layer': '350mb', 'rh': 5}","{'layer': '300mb', 'rh': 11}","{'layer': '250mb', 'rh': 6}","{'layer': '200mb', 'rh': -2}"
1,"{'layer': '950mb', 'rh': 3}","{'layer': '900mb', 'rh': 5}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 8}","{'layer': '750mb', 'rh': 8}","{'layer': '700mb', 'rh': 10}","{'layer': '650mb', 'rh': 10}","{'layer': '600mb', 'rh': 9}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 5}","{'layer': '450mb', 'rh': 10}","{'layer': '400mb', 'rh': 9}","{'layer': '350mb', 'rh': 9}","{'layer': '300mb', 'rh': 7}","{'layer': '250mb', 'rh': 4}","{'layer': '200mb', 'rh': -3}"
2,"{'layer': '950mb', 'rh': 4}","{'layer': '900mb', 'rh': 4}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 8}","{'layer': '750mb', 'rh': 11}","{'layer': '700mb', 'rh': 10}","{'layer': '650mb', 'rh': 8}","{'layer': '600mb', 'rh': 8}","{'layer': '550mb', 'rh': 7}","{'layer': '500mb', 'rh': 8}","{'layer': '450mb', 'rh': 7}","{'layer': '400mb', 'rh': 3}","{'layer': '350mb', 'rh': 2}","{'layer': '300mb', 'rh': 3}","{'layer': '250mb', 'rh': 6}","{'layer': '200mb', 'rh': -3}"
3,"{'layer': '950mb', 'rh': 5}","{'layer': '900mb', 'rh': 6}","{'layer': '850mb', 'rh': 7}","{'layer': '800mb', 'rh': 9}","{'layer': '750mb', 'rh': 11}","{'layer': '700mb', 'rh': 13}","{'layer': '650mb', 'rh': 10}","{'layer': '600mb', 'rh': 8}","{'layer': '550mb', 'rh': 8}","{'layer': '500mb', 'rh': 7}","{'layer': '450mb', 'rh': 4}","{'layer': '400mb', 'rh': 3}","{'layer': '350mb', 'rh': 2}","{'layer': '300mb', 'rh': 6}","{'layer': '250mb', 'rh': 12}","{'layer': '200mb', 'rh': -3}"
4,"{'layer': '950mb', 'rh': 6}","{'layer': '900mb', 'rh': 4}","{'layer': '850mb', 'rh': 7}","{'layer': '800mb', 'rh': 10}","{'layer': '750mb', 'rh': 13}","{'layer': '700mb', 'rh': 13}","{'layer': '650mb', 'rh': 10}","{'layer': '600mb', 'rh': 9}","{'layer': '550mb', 'rh': 8}","{'layer': '500mb', 'rh': 6}","{'layer': '450mb', 'rh': 4}","{'layer': '400mb', 'rh': 4}","{'layer': '350mb', 'rh': 8}","{'layer': '300mb', 'rh': 4}","{'layer': '250mb', 'rh': 6}","{'layer': '200mb', 'rh': -3}"


In [13]:
# Queremos extraer de los diccionarios el nombre de cada columna y el value de cada columna y fila.
# Hacemos un for loop. Aportamos el proceso razonado al final del jupyter.

df_limpio_layers = pd.DataFrame()

for col in range(len(df_rh_layer.columns)):
    # Volvemos a aplicar pd.Series a cada fila, ya que convertirá los keys en columnas y los values en filas 
    # y es una de las maneras posibles para extraer su contenido
    desempaque_dicc = df_rh_layer[col].apply(pd.Series)
    
    layer = desempaque_dicc.iloc[0].index[0]
    num = desempaque_dicc.iloc[0].values[0]
    #Construimos el nombre de la columna
    nom = 'rh_'+ layer + '_' + num

    # Creamos una lista que será el contenido de cada columna creada en este loop   
    lista = []
    for fila in range(len(desempaque_dicc.index)):
        
        value = desempaque_dicc.iloc[fila].values[1]
        lista.append(value)

    # Terminamos el loop rellenando nuestro df vacío con el contenido extraído
    df_limpio_layers[nom] = lista

In [14]:
# Reseteamos el indice para el siguiente paso
df_limpio_layers.reset_index(inplace=True)
df_limpio_layers.head()

,index,rh_layer_950mb,rh_layer_900mb,rh_layer_850mb,rh_layer_800mb,rh_layer_750mb,rh_layer_700mb,rh_layer_650mb,rh_layer_600mb,rh_layer_550mb,rh_layer_500mb,rh_layer_450mb,rh_layer_400mb,rh_layer_350mb,rh_layer_300mb,rh_layer_250mb,rh_layer_200mb
0,0,4,5,6,7,8,11,11,6,3,1,6,7,5,11,6,-2
1,1,3,5,6,8,8,10,10,9,5,5,10,9,9,7,4,-3
2,2,4,4,6,8,11,10,8,8,7,8,7,3,2,3,6,-3
3,3,5,6,7,9,11,13,10,8,8,7,4,3,2,6,12,-3
4,4,6,4,7,10,13,13,10,9,8,6,4,4,8,4,6,-3


##### Mergeamos con el df_clima

In [15]:
# Hacemos lo mismo con el df clima
df_clima.reset_index(inplace=True)

In [16]:
# Mergeamos por los indices
df_clima2 = df_clima.merge(df_limpio_layers, on= ['index'] )
df_clima2.head(1)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud,rh_layer_950mb,rh_layer_900mb,rh_layer_850mb,rh_layer_800mb,rh_layer_750mb,rh_layer_700mb,rh_layer_650mb,rh_layer_600mb,rh_layer_550mb,rh_layer_500mb,rh_layer_450mb,rh_layer_400mb,rh_layer_350mb,rh_layer_300mb,rh_layer_250mb,rh_layer_200mb
0,0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 4}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 155, 'speed':...",27,2,2,1011,none,0,0,190,2,usa,39.78373,-100.445882,4,5,6,7,8,11,11,6,3,1,6,7,5,11,6,-2


In [17]:
# Eliminamos la columna rh_profile
df_clima2.drop(['rh_profile'], axis=1, inplace=True)
df_clima2.head(1)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud,rh_layer_950mb,rh_layer_900mb,rh_layer_850mb,rh_layer_800mb,rh_layer_750mb,rh_layer_700mb,rh_layer_650mb,rh_layer_600mb,rh_layer_550mb,rh_layer_500mb,rh_layer_450mb,rh_layer_400mb,rh_layer_350mb,rh_layer_300mb,rh_layer_250mb,rh_layer_200mb
0,0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'direction': 155, 'speed':...",27,2,2,1011,none,0,0,190,2,usa,39.78373,-100.445882,4,5,6,7,8,11,11,6,3,1,6,7,5,11,6,-2


##### Vamos con la columna `wind_profile`

In [18]:
# Aplicamos pd.Series a cada fila
# Vemos que en vez de tener dos variables tenemos 3
df_wind = df_clima['wind_profile'].apply(pd.Series)
df_wind.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 155, 'speed': 2}","{'layer': '900mb', 'direction': 145, 'speed': 3}","{'layer': '850mb', 'direction': 160, 'speed': 3}","{'layer': '800mb', 'direction': 230, 'speed': 2}","{'layer': '750mb', 'direction': 250, 'speed': 3}","{'layer': '700mb', 'direction': 260, 'speed': 3}","{'layer': '650mb', 'direction': 250, 'speed': 4}","{'layer': '600mb', 'direction': 240, 'speed': 4}","{'layer': '550mb', 'direction': 235, 'speed': 5}","{'layer': '500mb', 'direction': 235, 'speed': 5}","{'layer': '450mb', 'direction': 235, 'speed': 5}","{'layer': '400mb', 'direction': 235, 'speed': 5}","{'layer': '350mb', 'direction': 230, 'speed': 5}","{'layer': '300mb', 'direction': 235, 'speed': 6}","{'layer': '250mb', 'direction': 230, 'speed': 7}","{'layer': '200mb', 'direction': 235, 'speed': 7}"
1,"{'layer': '950mb', 'direction': 155, 'speed': 2}","{'layer': '900mb', 'direction': 185, 'speed': 2}","{'layer': '850mb', 'direction': 230, 'speed': 2}","{'layer': '800mb', 'direction': 235, 'speed': 3}","{'layer': '750mb', 'direction': 250, 'speed': 3}","{'layer': '700mb', 'direction': 255, 'speed': 3}","{'layer': '650mb', 'direction': 240, 'speed': 3}","{'layer': '600mb', 'direction': 230, 'speed': 4}","{'layer': '550mb', 'direction': 235, 'speed': 5}","{'layer': '500mb', 'direction': 235, 'speed': 5}","{'layer': '450mb', 'direction': 240, 'speed': 5}","{'layer': '400mb', 'direction': 245, 'speed': 5}","{'layer': '350mb', 'direction': 250, 'speed': 5}","{'layer': '300mb', 'direction': 250, 'speed': 6}","{'layer': '250mb', 'direction': 240, 'speed': 6}","{'layer': '200mb', 'direction': 250, 'speed': 6}"
2,"{'layer': '950mb', 'direction': 120, 'speed': 3}","{'layer': '900mb', 'direction': 145, 'speed': 2}","{'layer': '850mb', 'direction': 245, 'speed': 2}","{'layer': '800mb', 'direction': 260, 'speed': 3}","{'layer': '750mb', 'direction': 275, 'speed': 3}","{'layer': '700mb', 'direction': 270, 'speed': 3}","{'layer': '650mb', 'direction': 245, 'speed': 3}","{'layer': '600mb', 'direction': 240, 'speed': 4}","{'layer': '550mb', 'direction': 245, 'speed': 4}","{'layer': '500mb', 'direction': 250, 'speed': 5}","{'layer': '450mb', 'direction': 245, 'speed': 5}","{'layer': '400mb', 'direction': 245, 'speed': 5}","{'layer': '350mb', 'direction': 255, 'speed': 5}","{'layer': '300mb', 'direction': 250, 'speed': 5}","{'layer': '250mb', 'direction': 245, 'speed': 5}","{'layer': '200mb', 'direction': 255, 'speed': 6}"
3,"{'layer': '950mb', 'direction': 110, 'speed': 2}","{'layer': '900mb', 'direction': 105, 'speed': 2}","{'layer': '850mb', 'direction': 160, 'speed': 2}","{'layer': '800mb', 'direction': 230, 'speed': 2}","{'layer': '750mb', 'direction': 255, 'speed': 3}","{'layer': '700mb', 'direction': 275, 'speed': 4}","{'layer': '650mb', 'direction': 280, 'speed': 3}","{'layer': '600mb', 'direction': 245, 'speed': 3}","{'layer': '550mb', 'direction': 250, 'speed': 4}","{'layer': '500mb', 'direction': 245, 'speed': 4}","{'layer': '450mb', 'direction': 240, 'speed': 4}","{'layer': '400mb', 'direction': 245, 'speed': 4}","{'layer': '350mb', 'direction': 245, 'speed': 5}","{'layer': '300mb', 'direction': 245, 'speed': 5}","{'layer': '250mb', 'direction': 255, 'speed': 5}","{'layer': '200mb', 'direction': 255, 'speed': 6}"
4,"{'layer': '950mb', 'direction': 265, 'speed': 2}","{'layer': '900mb', 'direction': 35, 'speed': 2}","{'layer': '850mb', 'direction': 80, 'speed': 2}","{'layer': '800mb', 'direction': 250, 'speed': 2}","{'layer': '750mb', 'direction': 270, 'speed': 2}","{'layer': '700mb', 'direction': 270, 'speed': 3}","{'layer': '650mb', 'direction': 270, 'speed': 3}","{'layer': '600mb', 'direction': 250, 'speed': 3}","{'layer': '550mb', 'direction': 245, 'speed': 3}","{'layer': '500mb', 'direction': 240, 'speed': 3}","{'layer': '450mb', 'direction': 245, 'speed': 3}","{'layer': '400mb', 'direction': 245, 'speed': 4}","{'layer': '350mb', 'direction': 245, 'speed': 4}","{'layer': '300mb', 'direc

In [19]:
# Probando con un elemento se nos ocurre crear dos columnas por cada columna con nombres: layer 950mb direction y layer 950mb speed, etc.
df_wind[0].apply(pd.Series)

,layer,direction,speed
0,950mb,155,2
1,950mb,155,2
2,950mb,120,3
3,950mb,110,2
4,950mb,265,2
5,950mb,320,2
6,950mb,335,2
7,950mb,355,2
8,950mb,10,2
9,950mb,110,2


In [20]:
# Hacemos un for loop
df_limpio_wind = pd.DataFrame()

for col in range(len(df_wind.columns)):
    # Volvemos a aplicar pd.Series a cada fila, ya que convertirá los keys en columnas y los values en filas 
    desempaq_dicc = df_wind[col].apply(pd.Series)
    
    layer = desempaq_dicc.iloc[0,:].index[0]
    num = desempaq_dicc.iloc[0,:].values[0]
    direction = desempaq_dicc.iloc[0,:].index[1]
    speed = desempaq_dicc.iloc[0,:].index[2]
    
    #Construimos el nombre de las columnas
    nom1 = layer + num + '_' + direction
    nom2 = layer + num + '_' + speed

    # Creamos una lista que será el contenido de cada columna creada en este loop   
    lista1 = []
    lista2 = []
    for fila in range(len(desempaq_dicc.index)):

        value_dir = desempaq_dicc.iloc[fila,:].values[1]
        value_sp= desempaq_dicc.iloc[fila,:].values[2]
        
        lista1.append(value_dir)
        lista2.append(value_sp)

    # Terminamos el loop rellenando nuestro df vacío con el contenido extraído
    df_limpio_wind[nom1] = lista1
    df_limpio_wind[nom2] = lista2

In [21]:
# Reseteamos el índice para el siguiente paso
df_limpio_wind.reset_index(inplace=True)
df_limpio_wind.head()

,index,layer950mb_direction,layer950mb_speed,layer900mb_direction,layer900mb_speed,layer850mb_direction,layer850mb_speed,layer800mb_direction,layer800mb_speed,layer750mb_direction,layer750mb_speed,layer700mb_direction,layer700mb_speed,layer650mb_direction,layer650mb_speed,layer600mb_direction,layer600mb_speed,layer550mb_direction,layer550mb_speed,layer500mb_direction,layer500mb_speed,layer450mb_direction,layer450mb_speed,layer400mb_direction,layer400mb_speed,layer350mb_direction,layer350mb_speed,layer300mb_direction,layer300mb_speed,layer250mb_direction,layer250mb_speed,layer200mb_direction,layer200mb_speed
0,0,155,2,145,3,160,3,230,2,250,3,260,3,250,4,240,4,235,5,235,5,235,5,235,5,230,5,235,6,230,7,235,7
1,1,155,2,185,2,230,2,235,3,250,3,255,3,240,3,230,4,235,5,235,5,240,5,245,5,250,5,250,6,240,6,250,6
2,2,120,3,145,2,245,2,260,3,275,3,270,3,245,3,240,4,245,4,250,5,245,5,245,5,255,5,250,5,245,5,255,6
3,3,110,2,105,2,160,2,230,2,255,3,275,4,280,3,245,3,250,4,245,4,240,4,245,4,245,5,245,5,255,5,255,6
4,4,265,2,35,2,80,2,250,2,270,2,270,3,270,3,250,3,245,3,240,3,245,3,245,4,245,4,250,5,245,5,255,5


In [22]:
# Celebramos

##### Mergeamos con  el df_clima2

In [23]:
df_clima3 = df_clima2.merge(df_limpio_wind, on= ['index'])
df_clima3.head(1)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud,rh_layer_950mb,rh_layer_900mb,rh_layer_850mb,rh_layer_800mb,rh_layer_750mb,rh_layer_700mb,rh_layer_650mb,rh_layer_600mb,rh_layer_550mb,rh_layer_500mb,rh_layer_450mb,rh_layer_400mb,rh_layer_350mb,rh_layer_300mb,rh_layer_250mb,rh_layer_200mb,layer950mb_direction,layer950mb_speed,layer900mb_direction,layer900mb_speed,layer850mb_direction,layer850mb_speed,layer800mb_direction,layer800mb_speed,layer750mb_direction,layer750mb_speed,layer700mb_direction,layer700mb_speed,layer650mb_direction,layer650mb_speed,layer600mb_direction,layer600mb_speed,layer550mb_direction,layer550mb_speed,layer500mb_direction,layer500mb_speed,layer450mb_direction,layer450mb_speed,layer400mb_direction,layer400mb_speed,layer350mb_direction,layer350mb_speed,layer300mb_direction,layer300mb_speed,layer250mb_direction,layer250mb_speed,layer200mb_direction,layer200mb_speed
0,0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'direction': 155, 'speed':...",27,2,2,1011,none,0,0,190,2,usa,39.78373,-100.445882,4,5,6,7,8,11,11,6,3,1,6,7,5,11,6,-2,155,2,145,3,160,3,230,2,250,3,260,3,250,4,240,4,235,5,235,5,235,5,235,5,230,5,235,6,230,7,235,7


In [24]:
# Eliminamos la columna wind_profile e index
df_clima3.drop(['index', 'wind_profile'], axis=1, inplace=True)
df_clima3.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud,rh_layer_950mb,rh_layer_900mb,rh_layer_850mb,rh_layer_800mb,rh_layer_750mb,rh_layer_700mb,rh_layer_650mb,rh_layer_600mb,rh_layer_550mb,rh_layer_500mb,rh_layer_450mb,rh_layer_400mb,rh_layer_350mb,rh_layer_300mb,rh_layer_250mb,rh_layer_200mb,layer950mb_direction,layer950mb_speed,layer900mb_direction,layer900mb_speed,layer850mb_direction,layer850mb_speed,layer800mb_direction,layer800mb_speed,layer750mb_direction,layer750mb_speed,layer700mb_direction,layer700mb_speed,layer650mb_direction,layer650mb_speed,layer600mb_direction,layer600mb_speed,layer550mb_direction,layer550mb_speed,layer500mb_direction,layer500mb_speed,layer450mb_direction,layer450mb_speed,layer400mb_direction,layer400mb_speed,layer350mb_direction,layer350mb_speed,layer300mb_direction,layer300mb_speed,layer250mb_direction,layer250mb_speed,layer200mb_direction,layer200mb_speed
0,3,1,-9999,-9999,-9999,27,2,2,1011,none,0,0,190,2,usa,39.78373,-100.445882,4,5,6,7,8,11,11,6,3,1,6,7,5,11,6,-2,155,2,145,3,160,3,230,2,250,3,260,3,250,4,240,4,235,5,235,5,235,5,235,5,230,5,235,6,230,7,235,7


##### Agrupamos por paises y calculamos la media para cada columna

In [25]:
df_grupo_clima = df_clima3.groupby('pais').mean()
df_grupo_clima.head()

,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,latitud,longitud,rh_layer_950mb,rh_layer_900mb,rh_layer_850mb,rh_layer_800mb,rh_layer_750mb,rh_layer_700mb,rh_layer_650mb,rh_layer_600mb,rh_layer_550mb,rh_layer_500mb,rh_layer_450mb,rh_layer_400mb,rh_layer_350mb,rh_layer_300mb,rh_layer_250mb,rh_layer_200mb,layer950mb_direction,layer950mb_speed,layer900mb_direction,layer900mb_speed,layer850mb_direction,layer850mb_speed,layer800mb_direction,layer800mb_speed,layer750mb_direction,layer750mb_speed,layer700mb_direction,layer700mb_speed,layer650mb_direction,layer650mb_speed,layer600mb_direction,layer600mb_speed,layer550mb_direction,layer550mb_speed,layer500mb_direction,layer500mb_speed,layer450mb_direction,layer450mb_speed,layer400mb_direction,layer400mb_speed,layer350mb_direction,layer350mb_speed,layer300mb_direction,layer300mb_speed,layer250mb_direction,layer250mb_speed,layer200mb_direction,layer200mb_speed
pais,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
australia,97.5,4.187500,-9999.0,-9999.0,-9999.0,22.140625,2.500000,10.953125,1023.578125,3.234375,0.0,3.328125,-24.776109,134.755000,13.921875,11.203125,8.187500,5.906250,3.859375,1.828125,-0.968750,-1.984375,-2.171875,-1.796875,-0.734375,-0.281250,0.343750,2.468750,6.343750,3.671875,96.328125,3.546875,91.953125,3.281250,99.687500,2.953125,112.187500,2.875000,103.281250,2.796875,105.859375,2.828125,132.656250,2.750000,189.140625,2.750000,206.015625,3.218750,227.812500,3.390625,238.046875,4.000000,254.062500,4.500000,255.937500,4.765625,253.750000,5.203125,246.875000,5.750000,252.343750,6.375000
new zealand,97.5,6.781250,-9999.0,-9999.0,-9999.0,10.640625,10.125000,9.328125,1026.687500,2.453125,0.0,3.437500,-41.500083,172.834408,11.562500,12.515625,6.375000,3.375000,1.328125,0.312500,0.812500,0.109375,0.296875,0.656250,1.437500,1.578125,3.031250,3.281250,4.062500,0.000000,272.109375,3.734375,271.093750,3.703125,270.156250,3.765625,268.359375,3.828125,266.640625,3.906250,266.171875,3.937500,265.703125,3.953125,266.640625,4.000000,266.562500,4.093750,252.031250,4.343750,254.375000,4.453125,248.828125,4.656250,238.671875,5.015625,242.109375,5.203125,256.953125,5.359375,265.703125,5.703125
papua new guinea,97.5,2.609375,-9999.0,-9999.0,-9999.0,25.875000,1.859375,11.000000,1011.750000,2.828125,0.0,3.718750,-5.681607,144.248908,13.390625,11.484375,8.437500,5.625000,1.703125,-1.187500,-2.531250,-3.515625,-3.718750,-3.984375,-3.109375,-2.156250,-1.765625,-1.687500,-1.390625,-0.546875,105.781250,4.093750,101.406250,4.234375,98.437500,3.859375,96.640625,3.109375,84.218750,2.718750,73.515625,2.453125,123.437500,2.515625,122.343750,2.718750,108.359375,3.078125,100.859375,3.218750,126.171875,2.828125,141.328125,3.203125,210.937500,3.781250,265.000000,4.765625,278.750000,4.750000,269.296875,4.875000
south africa,97.5,7.453125,-9999.0,-9999.0,-9999.0,16.890625,3.453125,12.265625,1017.093750,5.453125,0.0,2.937500,-28.816624,24.991639,12.406250,11.843750,11.343750,10.578125,10.640625,9.046875,6.234375,4.000000,3.484375,5.796875,6.984375,8.828125,10.843750,7.984375,7.031250,4.656250,189.609375,3.125000,200.859375,3.109375,225.546875,3.109375,249.140625,3.343750,257.500000,3.421875,268.984375,3.796875,268.437500,4.328125,269.218750,5.046875,270.000000,5.734375,275.312500,6.390625,275.468750,6.718750,270.937500,7.296875,275.312500,8.437500,272.031250,9.515625,269.531250,10.468750,267.109375,10.437500
usa,97.5,5.203125,-9999.0,-9999.0,-9999.0,24.671875,-0.125000,7.140625,1014.218750,2.234375,0.0,2.171875,39.783730,-100.445882,9.765625,11.578125,11.296875,10.031250,7.968750,5.671875,4.125000,3.765625,3.437500,3.906250,4.484375,6.625000,8.296875,9.093750,9.281250,2.453125,171.640625,2.109375,157.500000,2.234375,172.890625,2.203125,226.406250,2.421875,238.828125,2.765625,230.546875,2.953125,208.046875,3.046875,198.906250,3.093750,205.156250,3.203125,230.781250,3.281250,253.359375,3.375000,240.078125,3.703125,236.718

In [26]:
# Estamos listas para mergear con el df paises con la informacion de los ataques de tiburón

##### Unimos los df clima3 y df paises

In [27]:
# Vemos el nombre de las columnas por las que vamos a mergear
df_paises.columns

Index(['year', 'type', 'country', 'age', 'species_', 'fecha_limpia', 'fatal',
       'sex', 'latitud', 'longitud', 'country2', 'fatal_N', 'fatal_Unknown',
       'fatal_Y', 'fatal_N.1', 'fatal_Unknown.1', 'fatal_Y.1', 'species_.1',
       'fecha_limpia.1', 'type.1', 'age_NORM'],
      dtype='object')

In [28]:
df_grupo_clima.columns

Index(['timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'temp2m', 'lifted_index', 'rh2m', 'msl_pressure', 'prec_amount',
       'snow_depth', 'wind10m.speed', 'latitud', 'longitud', 'rh_layer_950mb',
       'rh_layer_900mb', 'rh_layer_850mb', 'rh_layer_800mb', 'rh_layer_750mb',
       'rh_layer_700mb', 'rh_layer_650mb', 'rh_layer_600mb', 'rh_layer_550mb',
       'rh_layer_500mb', 'rh_layer_450mb', 'rh_layer_400mb', 'rh_layer_350mb',
       'rh_layer_300mb', 'rh_layer_250mb', 'rh_layer_200mb',
       'layer950mb_direction', 'layer950mb_speed', 'layer900mb_direction',
       'layer900mb_speed', 'layer850mb_direction', 'layer850mb_speed',
       'layer800mb_direction', 'layer800mb_speed', 'layer750mb_direction',
       'layer750mb_speed', 'layer700mb_direction', 'layer700mb_speed',
       'layer650mb_direction', 'layer650mb_speed', 'layer600mb_direction',
       'layer600mb_speed', 'layer550mb_direction', 'layer550mb_speed',
       'layer500mb_direction', 'layer500mb

In [29]:
df_attacks_clima_paises = df_paises.merge(df_grupo_clima, left_on=['country'] , right_on=['pais'])
df_attacks_clima_paises.head()

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud_x,longitud_x,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed,latitud_y,longitud_y,rh_layer_950mb,rh_layer_900mb,rh_layer_850mb,rh_layer_800mb,rh_layer_750mb,rh_layer_700mb,rh_layer_650mb,rh_layer_600mb,rh_layer_550mb,rh_layer_500mb,rh_layer_450mb,rh_layer_400mb,rh_layer_350mb,rh_layer_300mb,rh_layer_250mb,rh_layer_200mb,layer950mb_direction,layer950mb_speed,layer900mb_direction,layer900mb_speed,layer850mb_direction,layer850mb_speed,layer800mb_direction,layer800mb_speed,layer750mb_direction,layer750mb_speed,layer700mb_direction,layer700mb_speed,layer650mb_direction,layer650mb_speed,layer600mb_direction,layer600mb_speed,layer550mb_direction,layer550mb_speed,layer500mb_direction,layer500mb_speed,layer450mb_direction,layer450mb_speed,layer400mb_direction,layer400mb_speed,layer350mb_direction,layer350mb_speed,layer300mb_direction,layer300mb_speed,layer250mb_direction,layer250mb_speed,layer200mb_direction,layer200mb_speed
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327,97.5,5.203125,-9999.0,-9999.0,-9999.0,24.671875,-0.125,7.140625,1014.21875,2.234375,0.0,2.171875,39.78373,-100.445882,9.765625,11.578125,11.296875,10.03125,7.96875,5.671875,4.125,3.765625,3.4375,3.90625,4.484375,6.625,8.296875,9.09375,9.28125,2.453125,171.640625,2.109375,157.5,2.234375,172.890625,2.203125,226.40625,2.421875,238.828125,2.765625,230.546875,2.953125,208.046875,3.046875,198.90625,3.09375,205.15625,3.203125,230.78125,3.28125,253.359375,3.375,240.078125,3.703125,236.71875,3.890625,236.40625,4.5,250.859375,5.015625,281.09375,5.359375
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860,97.5,5.203125,-9999.0,-9999.0,-9999.0,24.671875,-0.125,7.140625,1014.21875,2.234375,0.0,2.171875,39.78373,-100.445882,9.765625,11.578125,11.296875,10.03125,7.96875,5.671875,4.125,3.765625,3.4375,3.90625,4.484375,6.625,8.296875,9.09375,9.28125,2.453125,171.640625,2.109375,157.5,2.234375,172.890625,2.203125,226.40625,2.421875,238.828125,2.765625,230.546875,2.953125,208.046875,3.046875,198.90625,3.09375,205.15625,3.203125,230.78125,3.28125,253.359375,3.375,240.078125,3.703125,236.71875,3.890625,236.40625,4.5,250.859375,5.015625,281.09375,5.359375
2,2018,Invalid,usa,48.0,Unespecific,Jun,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,3,8.403629,97.5,5.203125,-9999.0,-9999.0,-9999.0,24.671875,-0.125,7.140625,1014.21875,2.234375,0.0,2.171875,39.78373,-100.445882,9.765625,11.578125,11.296875,10.03125,7.96875,5.671875,4.125,3.765625,3.4375,3.90625,4.484375,6.625,8.296875,9.09375,9.28125,2.453125,171.640625,2.109375,157.5,2.234375,172.890625,2.203125,226.40625,2.421875,238.828125,2.765625,230.546875,2.953125,208.046875,3.046875,198.90625,3.09375,205.15625,3.203125,230.78125,3.28125,253.359375,3.375,240.078125,3.703125,236.71875,3.890625,236.40625,4.5,250.859375,5.015625,281.09375,5.359375
3,2018,Unprovoked,usa,52.0,Lemon shark,May,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,1,9,7,8.730109,97.5,5.203125,-9999.0,-9999.0,-9999.0,24.671875,-0.125,7.140625,1014.21875,2.234375,0.0,2.171875,39.78373,-100.445882,9.765625,11.578125,11.296875,10.03125,7.96875,5.671875,4.125,3.765625,3.4375,3.90625,4.484375,6.625,8.296875,9.09375,9.28125,2.453125,171.640625,2.109375,157.5,2.234375,172.890625,2.203125,226.40625,2.421875,238.828125,2.765625,230.546875,2.953125,208.046875,3.046875,198.90625,3.09375,205.15625,3.203125,230.78125,3.28125,253.359375,3.375,240.078125,3.703125,236.71875,3.890625,236.40625,4.5,250.859375,5.015625,281.09375,5.359375
4,2018,Unprovoked,usa,15.0,Lemon shark,May,N,M,39.78373,-100.445882,HN,1,0,0,1,0,0,1,9,7,4.581659,97.5,5.203125,-9999.0,-9999.0,-9999.0,24.671875,-0.125,7.140625,1014.21875,2.234375,0.0,2.171875,39.78373,-100.445882,9.765625,11.

In [30]:
# Comprobamos nulos
df_attacks_clima_paises.isnull().sum()

year                    0
type                    0
country                 0
age                     0
species_                0
fecha_limpia            0
fatal                   0
sex                     0
latitud_x               0
longitud_x              0
country2                0
fatal_N                 0
fatal_Unknown           0
fatal_Y                 0
fatal_N.1               0
fatal_Unknown.1         0
fatal_Y.1               0
species_.1              0
fecha_limpia.1          0
type.1                  0
age_NORM                0
timepoint               0
cloudcover              0
highcloud               0
midcloud                0
lowcloud                0
temp2m                  0
lifted_index            0
rh2m                    0
msl_pressure            0
prec_amount             0
snow_depth              0
wind10m.speed           0
latitud_y               0
longitud_y              0
rh_layer_950mb          0
rh_layer_900mb          0
rh_layer_850mb          0
rh_layer_800

In [31]:
df_attacks_clima_paises['rh_layer_950mb'].isnull().sum()

0

In [32]:
df_attacks_clima_paises.shape

(4335, 83)

In [33]:
df_attacks_clima_paises['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [34]:
# Mergeo correcto

##### Guardamos en csv y en pickle

In [35]:
df_attacks_clima_paises.to_csv('datos/output/attacks_clima_paises.csv')

In [36]:
df_attacks_clima_paises.to_pickle('datos/output/attacks_clima_paises.pkl')

##### **Razonamiento detrás del for loop para la columna `wind_profile` (aplicado tambien para la columna rh_profile)

In [37]:
df_wind.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': 155, 'speed': 2}","{'layer': '900mb', 'direction': 145, 'speed': 3}","{'layer': '850mb', 'direction': 160, 'speed': 3}","{'layer': '800mb', 'direction': 230, 'speed': 2}","{'layer': '750mb', 'direction': 250, 'speed': 3}","{'layer': '700mb', 'direction': 260, 'speed': 3}","{'layer': '650mb', 'direction': 250, 'speed': 4}","{'layer': '600mb', 'direction': 240, 'speed': 4}","{'layer': '550mb', 'direction': 235, 'speed': 5}","{'layer': '500mb', 'direction': 235, 'speed': 5}","{'layer': '450mb', 'direction': 235, 'speed': 5}","{'layer': '400mb', 'direction': 235, 'speed': 5}","{'layer': '350mb', 'direction': 230, 'speed': 5}","{'layer': '300mb', 'direction': 235, 'speed': 6}","{'layer': '250mb', 'direction': 230, 'speed': 7}","{'layer': '200mb', 'direction': 235, 'speed': 7}"
1,"{'layer': '950mb', 'direction': 155, 'speed': 2}","{'layer': '900mb', 'direction': 185, 'speed': 2}","{'layer': '850mb', 'direction': 230, 'speed': 2}","{'layer': '800mb', 'direction': 235, 'speed': 3}","{'layer': '750mb', 'direction': 250, 'speed': 3}","{'layer': '700mb', 'direction': 255, 'speed': 3}","{'layer': '650mb', 'direction': 240, 'speed': 3}","{'layer': '600mb', 'direction': 230, 'speed': 4}","{'layer': '550mb', 'direction': 235, 'speed': 5}","{'layer': '500mb', 'direction': 235, 'speed': 5}","{'layer': '450mb', 'direction': 240, 'speed': 5}","{'layer': '400mb', 'direction': 245, 'speed': 5}","{'layer': '350mb', 'direction': 250, 'speed': 5}","{'layer': '300mb', 'direction': 250, 'speed': 6}","{'layer': '250mb', 'direction': 240, 'speed': 6}","{'layer': '200mb', 'direction': 250, 'speed': 6}"
2,"{'layer': '950mb', 'direction': 120, 'speed': 3}","{'layer': '900mb', 'direction': 145, 'speed': 2}","{'layer': '850mb', 'direction': 245, 'speed': 2}","{'layer': '800mb', 'direction': 260, 'speed': 3}","{'layer': '750mb', 'direction': 275, 'speed': 3}","{'layer': '700mb', 'direction': 270, 'speed': 3}","{'layer': '650mb', 'direction': 245, 'speed': 3}","{'layer': '600mb', 'direction': 240, 'speed': 4}","{'layer': '550mb', 'direction': 245, 'speed': 4}","{'layer': '500mb', 'direction': 250, 'speed': 5}","{'layer': '450mb', 'direction': 245, 'speed': 5}","{'layer': '400mb', 'direction': 245, 'speed': 5}","{'layer': '350mb', 'direction': 255, 'speed': 5}","{'layer': '300mb', 'direction': 250, 'speed': 5}","{'layer': '250mb', 'direction': 245, 'speed': 5}","{'layer': '200mb', 'direction': 255, 'speed': 6}"
3,"{'layer': '950mb', 'direction': 110, 'speed': 2}","{'layer': '900mb', 'direction': 105, 'speed': 2}","{'layer': '850mb', 'direction': 160, 'speed': 2}","{'layer': '800mb', 'direction': 230, 'speed': 2}","{'layer': '750mb', 'direction': 255, 'speed': 3}","{'layer': '700mb', 'direction': 275, 'speed': 4}","{'layer': '650mb', 'direction': 280, 'speed': 3}","{'layer': '600mb', 'direction': 245, 'speed': 3}","{'layer': '550mb', 'direction': 250, 'speed': 4}","{'layer': '500mb', 'direction': 245, 'speed': 4}","{'layer': '450mb', 'direction': 240, 'speed': 4}","{'layer': '400mb', 'direction': 245, 'speed': 4}","{'layer': '350mb', 'direction': 245, 'speed': 5}","{'layer': '300mb', 'direction': 245, 'speed': 5}","{'layer': '250mb', 'direction': 255, 'speed': 5}","{'layer': '200mb', 'direction': 255, 'speed': 6}"
4,"{'layer': '950mb', 'direction': 265, 'speed': 2}","{'layer': '900mb', 'direction': 35, 'speed': 2}","{'layer': '850mb', 'direction': 80, 'speed': 2}","{'layer': '800mb', 'direction': 250, 'speed': 2}","{'layer': '750mb', 'direction': 270, 'speed': 2}","{'layer': '700mb', 'direction': 270, 'speed': 3}","{'layer': '650mb', 'direction': 270, 'speed': 3}","{'layer': '600mb', 'direction': 250, 'speed': 3}","{'layer': '550mb', 'direction': 245, 'speed': 3}","{'layer': '500mb', 'direction': 240, 'speed': 3}","{'layer': '450mb', 'direction': 245, 'speed': 3}","{'layer': '400mb', 'direction': 245, 'speed': 4}","{'layer': '350mb', 'direction': 245, 'speed': 4}","{'layer': '300mb', 'direc

In [38]:
# Desempaquetamos una columna
des_dicc = df_wind[0].apply(pd.Series)
des_dicc

,layer,direction,speed
0,950mb,155,2
1,950mb,155,2
2,950mb,120,3
3,950mb,110,2
4,950mb,265,2
5,950mb,320,2
6,950mb,335,2
7,950mb,355,2
8,950mb,10,2
9,950mb,110,2


In [39]:
# Sacamos uno de los diccionarios
des_dicc.iloc[0,:]

layer        950mb
direction      155
speed            2
Name: 0, dtype: object

In [40]:
# Construimos el nombre de la columna
layer = des_dicc.iloc[0,:].index[0]
layer

'layer'

In [41]:
num = des_dicc.iloc[0,:].values[0]
num

'950mb'

In [42]:
direction = des_dicc.iloc[0,:].index[1]
direction

'direction'

In [43]:
speed = des_dicc.iloc[0,:].index[2]
speed

'speed'

In [44]:
# Sacamos los valores de cada fila
value_dir = des_dicc.iloc[0,:].values[1]
value_dir

155

In [45]:
value_sp= des_dicc.iloc[0,:].values[2]
value_sp

2

In [46]:
# Copiaremos todas estas variables para nuestro for loop para iterar por cada columna y dentro de cada columna extraer los valores de todas las filas
layer = des_dicc.iloc[0,:].index[0]
num = des_dicc.iloc[0,:].values[0]
direction = des_dicc.iloc[0,:].index[1]
speed = des_dicc.iloc[0,:].index[2]
value_dir = des_dicc.iloc[0,:].values[1]
value_sp= des_dicc.iloc[0,:].values[2]